# Importing the dependencies

In [5]:
import pandas as pd
import numpy as np
import sqlite3

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax



# Loading the Database
We will now load the database and work with some high end language models, and LLMs, to better understand the dataset and pre compute some of the features that we will use in the model.

In [ ]:
# data_query_based = pd.read_csv("../Data/Movies_Query_Based.csv")
# data_query_based.head()

In [2]:
data_query_based = pd.read_csv("/content/Movies_Query_Based.csv")
data_query_based.head()

,id,IMDB_ID,title,release_year,genres,vote_average,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Director,keywords,reviews,poster_path,backdrop_path
0,283995,tt3896198,Guardians of the Galaxy Vol. 2,2017,Adventure Action Sci-Fi,7.623,Chris Pratt,Zoe Saldaña,Dave Bautista,Vin Diesel,Bradley Cooper,James Gunn,"['demi god', 'alien creature', 'sarcasm', 'cra...","[""Despite being a huge comic book nerd I was n...",/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg
1,480530,tt6343314,Creed II,2018,Drama,6.990,Michael B. Jordan,Sylvester Stallone,Tessa Thompson,Wood Harris,Russell Hornsby,Steven Caple Jr.,"['baby', 'training montage', 'sequel', 'boxing...","[""This movie is not as good as the first Creed...",/v3QyboWRoA4O9RbcsqH8tJMe8EB.jpg,/xTYGN1b3XkOtODryXTKgdXLtPMz.jpg
2,299536,tt4154756,Avengers: Infinity War,2018,Adventure Action Sci-Fi,8.260,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson,Anthony RussoJoe Russo,"['superhero', 'ensemble cast', 'marvel cinemat...","[""Avengers infinity war is an emotional roller...",/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg
3,299534,tt4154796,Avengers: Endgame,2019,Adventure Sci-Fi Action,8.268,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Scarlett Johansson,Anthony RussoJoe Russo,"['time travel', 'superhero', 'super villain', ...","[""But its a pretty good film. A bit of a mess ...",/or06FN3Dka5tukK1e9sl16pB3iy.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg
4,337167,tt4477536,Fifty Shades Freed,2018,Drama Romance,6.699,Dakota Johnson,Jamie Dornan,Eric Johnson,Luke Grimes,Rita Ora,James Foley,"['sex scene', 'wedding ceremony', 'bondage', '...","[""The first of the three that is actually emot...",/9ZedQHPQVveaIYmDSTazhT3y273.jpg,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg


# Sentiment Analysis with ROBERTA
As we have seen in the previous notebook [here](./Sentiment_Model.ipynb), the sentiment analysis done using ROBERTA has proved to be the most efficient to correctly predict the sentiment of the text. We will now use the same model to predict the sentiment of the text in the dataset, and store them as values.

In [3]:
MOD = f"aychang/roberta-base-imdb"
tokenizer = AutoTokenizer.from_pretrained(MOD)
model = AutoModelForSequenceClassification.from_pretrained(MOD).to("cuda")

In [22]:
def sentiment_pred_roberta(text):
    encoded_text = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    output = model(**encoded_text)

    scores = output[0][0].detach().numpy()
    
    scores = softmax(scores)

    max_index = np.argmax(scores)

    if max_index == 0:
        sentiment = 'negative'
    elif max_index == 1:
        sentiment = 'positive'

    return sentiment

In [29]:
list1 = []
for i in eval(data_query_based['reviews'].iloc[0]):
    list1.append(sentiment_pred_roberta(i))

list1

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive']